In [1]:
import numpy as np

In [2]:
import sys
import time

In [3]:
sys.path.append('../LDPC_lib')

In [16]:
import Turbo_lib
import _Turbo_lib
from rew import AWGN

In [5]:
F = Turbo_lib.FSM()

In [6]:
F.test()

Nerr uncoded:0, 	 Nerr coded:0, 	 Time ref:0.000758
Nerr coded2:0, 	 Time coded2:0.001329


(0, 0)

In [7]:
F.__dict__

{'S': array([[0, 1],
        [1, 0]]),
 'Q': array([[0, 1],
        [1, 0]]),
 'Md': 2,
 'Ms': 2,
 'K': 1,
 'Mc': 2}

In [27]:
l = 5000000
SNR = 3
d = np.random.randint(2, size=l)
c = F.encode(d)
s = 2*c - 1 # BPSK
(x, sigma2w) = AWGN(s, SNR)
mu = np.zeros([l, 2])
mu[:,0] = np.exp(-np.abs(x + 1)**2/sigma2w)
mu[:,1] = np.exp(-np.abs(x - 1)**2/sigma2w)

pd = np.ones(l*2, float)/2  # no a priory knowledge
pc = mu.flatten() # observation


In [28]:
out = np.zeros(pd.shape)

In [15]:
F.update_both(pc, pd)

(array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]),
 array([8.86141056e-04, 9.99113859e-01, 9.80849954e-01, 1.91500463e-02,
        1.97454962e-02, 9.80254504e-01, 1.09842603e-02, 9.89015740e-01,
        9.90455173e-01, 9.54482656e-03]))

In [34]:
out = np.zeros_like(mu)

In [35]:
out.shape

(5000000, 2)

In [50]:
def update_general_FSM_python(pc, pd, out, S, Q, Md, Ms, Mc, N):
    lf = np.zeros([N + 1, Ms], dtype=float)
    lb = np.zeros([N + 1, Ms], dtype=float)
    
    lf[0,0] = 1;
    for i in range(1, Ms):
        lf[0,i] = 0 # 0 zero initial state

    for i in range(Ms):
        lb[N,i] = 1./Ms # unknown final state
        
    for i in range(N): # Forward
        for j in range(Ms):
            for k in range(Md):
                lf[i + 1, S[k, j]] += lf[i, j] * pd[i,k]*pc[i,Q[k,j]]
        m = 0
        for j in range(Ms):
            m += lf[i + 1, j]
        m = 1./m
        for j in range(Ms):
            lf[i + 1, j] = lf[i + 1, j] * m

#    lf[i + 1, :] = lf[i + 1, :] / (lf[i + 1, :]).sum()  # Norming

    for index in range(N): # Backward
        for j in range(Ms):
            for k in range(Md):
                i = N - 1 - index
                lb[i, j] += lb[i + 1, S[k, j]] * pc[i ,Q[k, j]] * pd[i, k]
            m = 0
            for j in range(Ms):
                m += lb[i, j]
            m = 1./m
        for j in range(Ms):
            lb[i, j] = lb[i, j] * m
            
    for i in range(N): # To data
        for j in range(Ms):
            for k in range(Md):
                out[i, k] += lb[i+1, S[k, j]] * pc[i, Q[k, j]] * lf[i, j]
            m = 0
            for k in range(Md):
                m += out[i, k]
            m = 1./m
            for k in range(Md):
                out[i, k] = out[i, k] * m


In [53]:
#%timeit update_general_FSM_python(mu, mu, out, np.uint32(F.S), np.uint32(F.Q),np.uint32(2),np.uint32(2),np.uint32(2), np.uint32(len(mu)))


In [54]:
%load_ext pythran.magic

In [62]:
def update_general_FSM_python(pc, pd, out, S, Q, Md, Ms, Mc, N):
    lf = np.zeros([N + 1, Ms], dtype=float)
    lb = np.zeros([N + 1, Ms], dtype=float)
    
    lf[0,0] = 1;
    for i in range(1, Ms):
        lf[0,i] = 0 # 0 zero initial state

    for i in range(Ms):
        lb[N,i] = 1./Ms # unknown final state
        
    for i in range(N): # Forward
        for j in range(Ms):
            for k in range(Md):
                lf[i + 1, S[k, j]] += lf[i, j] * pd[i,k]*pc[i,Q[k,j]]
        m = 0
        for j in range(Ms):
            m += lf[i + 1, j]
        m = 1./m
        for j in range(Ms):
            lf[i + 1, j] = lf[i + 1, j] * m

#    lf[i + 1, :] = lf[i + 1, :] / (lf[i + 1, :]).sum()  # Norming

    for index in range(N): # Backward
        for j in range(Ms):
            for k in range(Md):
                i = N - 1 - index
                lb[i, j] += lb[i + 1, S[k, j]] * pc[i ,Q[k, j]] * pd[i, k]
            m = 0
            for j in range(Ms):
                m += lb[i, j]
            m = 1./m
        for j in range(Ms):
            lb[i, j] = lb[i, j] * m
            
    for i in range(N): # To data
        for j in range(Ms):
            for k in range(Md):
                out[i, k] += lb[i+1, S[k, j]] * pc[i, Q[k, j]] * lf[i, j]
            m = 0
            for k in range(Md):
                m += out[i, k]
            m = 1./m
            for k in range(Md):
                out[i, k] = out[i, k] * m
    return out

In [58]:
from numba import jit

In [61]:
@jit
def update_general_FSM_numba(pc, pd, out, S, Q, Md, Ms, Mc, N):
    lf = np.zeros([N + 1, Ms], dtype=float)
    lb = np.zeros([N + 1, Ms], dtype=float)
    
    lf[0,0] = 1;
    for i in range(1, Ms):
        lf[0,i] = 0 # 0 zero initial state

    for i in range(Ms):
        lb[N,i] = 1./Ms # unknown final state
        
    for i in range(N): # Forward
        for j in range(Ms):
            for k in range(Md):
                lf[i + 1, S[k, j]] += lf[i, j] * pd[i,k]*pc[i,Q[k,j]]
        m = 0
        for j in range(Ms):
            m += lf[i + 1, j]
        m = 1./m
        for j in range(Ms):
            lf[i + 1, j] = lf[i + 1, j] * m

#    lf[i + 1, :] = lf[i + 1, :] / (lf[i + 1, :]).sum()  # Norming

    for index in range(N): # Backward
        for j in range(Ms):
            for k in range(Md):
                i = N - 1 - index
                lb[i, j] += lb[i + 1, S[k, j]] * pc[i ,Q[k, j]] * pd[i, k]
            m = 0
            for j in range(Ms):
                m += lb[i, j]
            m = 1./m
        for j in range(Ms):
            lb[i, j] = lb[i, j] * m
            
    for i in range(N): # To data
        for j in range(Ms):
            for k in range(Md):
                out[i, k] += lb[i+1, S[k, j]] * pc[i, Q[k, j]] * lf[i, j]
            m = 0
            for k in range(Md):
                m += out[i, k]
            m = 1./m
            for k in range(Md):
                out[i, k] = out[i, k] * m


In [63]:
%timeit update_general_FSM_numba(mu, mu, out, np.uint32(F.S), np.uint32(F.Q),np.uint32(2),np.uint32(2),np.uint32(2), np.uint32(len(mu)))


<ipython-input-61-095c4e336809>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "update_general_FSM_numba" failed type inference due to: No implementation of function Function(<built-in function zeros>) found for signature:
 
 >>> zeros(list(int64)<iv=None>, dtype=Function(<class 'float'>))
 
There are 2 candidate implementations:
     - Of which 2 did not match due to:
     Overload of function 'zeros': File: numba/core/typing/npydecl.py: Line 504.
       With argument(s): '(list(int64)<iv=None>, dtype=Function(<class 'float'>))':
      No match.

During: resolving callee type: Function(<built-in function zeros>)
During: typing of call at <ipython-input-61-095c4e336809> (3)


File "<ipython-input-61-095c4e336809>", line 3:
def update_general_FSM_numba(pc, pd, out, S, Q, Md, Ms, Mc, N):
    lf = np.zeros([N + 1, Ms], dtype=float)
    ^

  @jit
<ipython-input-61-095c4e336809>:1: NumbaWarning: 
Compilation is falling back to object m

KeyboardInterrupt: 

In [39]:
%timeit _Turbo_lib.update_general_FSM_C(mu, mu, out, np.uint32(F.S), np.uint32(F.Q),np.uint32(2),np.uint32(2),np.uint32(2), np.uint32(len(mu)))


445 ms ± 5.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%timeit F.update(pc, pd)

257 ms ± 2.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
r = %prun -r _Turbo_lib.update_general_FSM_C(mu, mu, out, np.uint32(F.S), np.uint32(F.Q),np.uint32(2),np.uint32(2),np.uint32(2), np.uint32(len(mu)))

r.print_stats()

          5 function calls in 0.465 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.465    0.465    0.465    0.465 {_Turbo_lib.update_general_FSM_C}
        1    0.000    0.000    0.465    0.465 <string>:1(<module>)
        1    0.000    0.000    0.465    0.465 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




         5 function calls in 0.465 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.465    0.465    0.465    0.465 {_Turbo_lib.update_general_FSM_C}
        1    0.000    0.000    0.465    0.465 <string>:1(<module>)
        1    0.000    0.000    0.465    0.465 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [43]:
%load_ext line_profiler

In [45]:
r = %prun -r F.update(pc, pd)
r.print_stats()

          14 function calls in 0.288 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.286    0.286    0.286    0.286 {_Turbo_lib.update_general_FSM_func}
        1    0.002    0.002    0.288    0.288 <string>:1(<module>)
        1    0.000    0.000    0.287    0.287 Turbo_lib.py:49(update)
        1    0.000    0.000    0.288    0.288 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        1    0.000    0.000    0.000    0.000 fromnumeric.py:70(_wrapreduction)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2589(amax)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(amax)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.0

         14 function calls in 0.288 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.286    0.286    0.286    0.286 {_Turbo_lib.update_general_FSM_func}
        1    0.002    0.002    0.288    0.288 <string>:1(<module>)
        1    0.000    0.000    0.287    0.287 Turbo_lib.py:49(update)
        1    0.000    0.000    0.288    0.288 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        1    0.000    0.000    0.000    0.000 fromnumeric.py:70(_wrapreduction)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2589(amax)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(amax)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.00

In [47]:
%load_ext cython

In [48]:
%load_ext pythran.magic

ModuleNotFoundError: No module named 'pythran'

In [ ]:
%%pythran
#pythran export laplace_pythran_highlevel(float[][])
import numpy as np
def laplace_pythran_highlevel(image):
    """Laplace operator in NumPy for 2D images. Pythran accelerated."""
    laplacian = image[:-2, 1:-1] + image[2:, 1:-1] + image[1:-1, :-2] + image[1:-1, 2:] - 4*image[1:-1, 1:-1]
    thresh = np.abs(laplacian) > 0.05
    return thresh


In [ ]:
l = 5
SNR = 3
d = np.random.randint(2, size=l)
c = F.encode(d)
s = 2*c - 1 # BPSK
(x, sigma2w) = AWGN(s, SNR)
mu = np.zeros([l, 2])
mu[:,0] = np.exp(-np.abs(x + 1)**2/sigma2w)
mu[:,1] = np.exp(-np.abs(x - 1)**2/sigma2w)

pd = np.ones(l*2, float)/2  # no a priory knowledge
pc = mu.flatten() # observation
s_c = time.time()
res_conv = F.update(pc, pd).reshape(l, 2) # conventional prob based solution
e_c = time.time()
s_c2 = time.time()
res_conv2 = F.update_C(mu, np.ones([l,2],float)/2) # conventional prob based solution
e_c2 = time.time()
#print res_conv[:10], np.argmax(res_conv, axis=1)[:10], d[:10]
nerr_c2 = np.sum(np.argmax(res_conv2, axis=1) != d)
nerr_c = np.sum(np.argmax(res_conv, axis=1) != d)
nerr_u = np.sum(np.argmax(mu, axis=1) != c)

print('Nerr uncoded:%d, \t Nerr coded:%d, \t Time ref:%.6f'%(nerr_u, nerr_c, e_c-s_c))
print('Nerr coded2:%d, \t Time coded2:%.6f'%(nerr_c2, e_c2-s_c2))



In [1]:
run Advandeced_Turbo_lib.py

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(sf_inds, sf_vals)? (Advandeced_Turbo_lib.py, line 350)

In [5]:
SNR = 5
l = 20
S = np.zeros([2,8],int)
Q = np.zeros([2,8],int)
S1 = np.zeros([2,8],int)
Q1 = np.zeros([2,8],int)
Qm1 = np.zeros([2,8],int)
_Turbo_lib.create_matrix_SQ_XOR_all(S,Q,S1,Q1,Qm1)
F = advanced_FSM(S,Q,S1,Q1, Qm1)
l1 = l+F.K

In [6]:
d = np.random.randint(2, size=l)
#d=np.array([0, 1, 1, 0, 1, 1, 0, 0, 1, 1])
c = F.encode_zero_term(d)
s = 2*c - 1 # BPSK
(x, sigma2w) = AWGN(s, SNR)
mu = np.zeros([l1, 2])
mu[:,0] = np.exp(-np.abs(x + 1)**2/sigma2w)
mu[:,1] = np.exp(-np.abs(x - 1)**2/sigma2w)

pd = np.ones(l1*2, float)/2  # no a priory knowledge
pc = mu.flatten() # observation
res_conv = F.update(pc, pd).reshape(l1, 2) # conventional prob based solution
res_conv2 = F.update_C(mu, np.ones([l1,2],float)/2) # conventional prob based solution
#    print res_conv[:10], np.argmax(res_conv, axis=1)[:10], d[:10]
nerr_c2 = np.sum(np.argmax(res_conv2, axis=1) != d)
nerr_c = np.sum(np.argmax(res_conv, axis=1) != d)
nerr_u = np.sum(np.argmax(mu, axis=1) != c)

TypeError: unsupported operand type(s) for -: 'tuple' and 'int'

In [7]:
c

(array([0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0]),
 array([1, 1, 0]))

In [8]:
pc_inds = np.array([[0,1]]).repeat(l1,axis=0)
pd_inds = np.array([[0,1]]).repeat(l1,axis=0)
pd_vals = np.ones([l1,2])/2
pc_vals = mu

In [9]:
%time a =F.update_lim(pd_inds, pd_vals, pc_inds, pc_vals, 2,8,2)
print np.sum(np.abs(a[1][:-3,0]-d))

CPU times: user 32 ms, sys: 1 ms, total: 33 ms
Wall time: 33.3 ms
0


In [10]:
print a[1][:,0]
print d

[1 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1 0 1 1 0 0 1 0]
[1 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1 0 1 1 0]


In [11]:
%time a = F.update_lim(pd_inds, pd_vals, pc_inds, pc_vals, 2,2,2)

CPU times: user 17 ms, sys: 0 ns, total: 17 ms
Wall time: 16.5 ms


In [50]:
def pmf2df(pmf, max_vals):
    x,y = pmf.shape
    if y <= max_vals:
        inds = np.array([np.arange(y)]).repeat(x,axis=0)
        vals = pmf
    else:
        temp = np.argsort(pmf, axis = 1)[:,-1::-1]        
        vals = np.vstack([pmf[i,temp[i,:max_vals]] for i in range(x)])        
        inds = temp[:,:max_vals]
        
    return inds, vals


In [1]:
run Advandeced_Turbo_lib.py

In [2]:
def QAM(M): # 2**M QAM constellation
    if M == 0:
        return np.array([0])
    odd = M % 2
    base = (M - odd) / 2
    min_Re = -2**(base+odd) + 1
    max_Re = 2**(base+odd) - 1
    min_Im = -2**base + 1
    max_Im = 2**base - 1
    const = np.asarray([i + 1j*j  for i in np.arange(min_Re, max_Re+1, 2)\
                                   for j in np.arange(min_Im, max_Im+1, 2)])
    scale = np.sqrt((np.abs(const)**2).sum()/float(2**M))
    return const / scale


In [3]:
Nb = 4
k = 2
const = QAM(Nb)
SNR = 12
Aq = 2**Nb

In [4]:
S = Sparse_Turbo_Coder(k, 1./3, [], [], 3, Aq, 0, 5)

In [5]:
#b = np.random.randint(Aq, size=k)
b = np.array([12,3],int)
c = S.encode_zero(b)
print c
#print len(c)
s = const[c]
(x, sigma2w) = AWGN(s, SNR)

[12  3 12 15  3  3 15 12 12  3 15 15 15 15  3 15 15 12]


In [6]:
#S.F.encode(np.array([3,1]))
print S.F.encode_zero_term(np.array([3,1]))


(array([3, 2, 2, 2, 1]), array([3, 2, 1]))


In [7]:
b_int = b[S.inter]
k = len(b)
p = S.n - k
c1_full, b_add1 = S.F.encode_zero_term(b)
c2_full, b_add2 = S.F.encode_zero_term(b_int)
c1 = c1_full[:(-S.F.K)]
c2 = c2_full[:(-S.F.K)]
c1_add = c1_full[(-S.F.K):]
c2_add = c2_full[(-S.F.K):]
print b, b_add1, b_add2, c1_add, c2_add, c1, c2

[12  3] [12 15  3] [ 3 15 12] [15 15  3] [15 15 12] [12 15] [ 3 15]


In [10]:
plt.plot(np.real(x), np.imag(x), 'xr')
plt.plot(np.real(const), np.imag(const), 'ob')
plt.show()

In [8]:
N = len(c)

In [9]:
mu = np.zeros([N, Aq])
for i in range(Aq): 
    mu[:, i] = np.exp(-np.abs(x - const[i])**2/sigma2w)


In [23]:
#%time inds, vals = S.decode(mu, max_vals = 16, max_valsS = 100)

In [16]:

def decide(inds, vals):
    return [inds[i,np.argmax(vals[i,:])] for i in range(len(inds))]

print decide(inds,vals)- b
    

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]


In [15]:
mu

array([[  6.17077428e-20,   3.37552430e-01,   8.03018510e-35,
          4.39265540e-16],
       [  1.66330203e-26,   1.16218885e-09,   2.63616434e-18,
          1.84195098e-01],
       [  9.90645875e-16,   9.26073659e-01,   4.06060603e-30,
          3.79592787e-15],
       [  1.37711278e-11,   6.51797197e-01,   3.52714051e-26,
          1.66942049e-15],
       [  1.75739676e-15,   3.47662041e-33,   5.12367199e-01,
          1.01360507e-18],
       [  1.52773676e-16,   2.17846883e-27,   6.08210683e-01,
          8.67275075e-12]])

In [10]:
max_vals = 16
k = S.k
K = S.F.K
p = S.p
inter = S.inter
deinter = S.deinter
mbi = mu[:k,:]
mci = mu[(k+K*2):(-2*K),:]
b1term_i, b1term_v= pmf2df(mu[k:(k+K), :], max_vals)
b2term_i, b2term_v= pmf2df(mu[(k+K):(k+2*K), :], max_vals)
c1term_i, c1term_v= pmf2df(mu[(-2*K):(-K), :], max_vals)
c2term_i, c2term_v = pmf2df(mu[(-K):, :], max_vals)
map_inds_c1 = S.map_inds_c1
map_inds_c2 = S.map_inds_c2
mci1t = np.ones([k, Aq], float)/Aq;
mci1t[map_inds_c1, :] = mci[::2]
mci2t = np.ones([k, Aq], float)/Aq;
mci2t[map_inds_c2, :] = mci[1::2]

b_inds, b_vals = pmf2df(mbi, max_vals)
c1_inds, c1_vals = pmf2df(mci1t, max_vals)
c2_inds, c2_vals = pmf2df(mci2t, max_vals)
Nd = np.min([max_vals, Aq])
Nc = np.min([max_vals, Aq])

c1_inds = np.vstack([c1_inds, c1term_i])
c1_vals = np.vstack([c1_vals, c1term_v])
c2_inds = np.vstack([c2_inds, c2term_i])
c2_vals = np.vstack([c2_vals, c2term_v])

in1_inds = b_inds
in1_vals = b_vals


In [11]:
in1_inds = np.vstack([in1_inds, b1term_i])
in1_vals = np.vstack([in1_vals, b1term_v])


Nd = Aq
Nc = Aq
max_valsS = 64
#out1_vals, out1_inds = S.F.update_lim(in1_inds, in1_vals, c1_inds, c1_vals, Nd, 50, Nc)
#temp = np.vstack([VN2_update(out1_inds[i,:], b_inds[i,:], out1_vals[i,:], b_vals[i,:]) for i in range(k)])
#in2_inds = temp[::2]
#in2_vals = temp[1::2]
#in2_inds = np.vstack([in2_inds[inter], b2term_i])
#in2_vals = np.vstack([in2_vals[inter], b2term_v])
#out2_vals, out2_inds = S.F.update_lim(in2_inds, in2_vals, c2_inds, c2_vals, Nd, 50, Nc)


#out2_vals, out2_inds = S.F.update_lim(in2_inds, in2_vals, c2_inds, c2_vals, Nd, 50, Nc)
#in1_inds = out2_inds[deinter]
#in1_vals = out2_vals[deinter]
#temp = np.vstack([VN2_update(in1_inds[i,:], b_inds[i,:], in1_vals[i,:], b_vals[i,:]) for i in range(k)])
#in1_inds = temp[::2]
#in1_vals = temp[1::2]

In [13]:
out1_vals, out1_inds = S.F.update_lim(in1_inds, in1_vals, c1_inds, c1_vals, 4, 8, 4)

[[ 0 -1  0  0  0  0  0  0]
 [16 48  0 32 -1  0  0  0]
 [52 20 36 60  4 48 28 44]
 [29 31  9 13 53 25 21 28]
 [51 35 19 50 49 59 54 38]
 [ 0 32  2 17 16  4 40 20]] [[  1.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   1.77167841e-21   2.30454247e-31   4.08290813e-52
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   1.25901855e-11   5.76150445e-13   1.77167841e-21
    7.25384097e-24   2.30454247e-31   2.23057598e-32   1.02075330e-33]
 [  1.00000000e+00   1.77167841e-21   3.18756203e-23   6.53382154e-26
    1.39061272e-26   6.80425258e-28   2.04034772e-28   2.30454247e-31]
 [  1.00000000e+00   2.40356055e-20   2.83569011e-25   6.80425258e-28
    2.04034772e-28   8.42608118e-32   5.59323489e-33   4.36625705e-33]
 [  1.00000000e+00   4.58497991e-24   8.42608118e-32   4.36625705e-33
    4.22347447e-33   9.30917244e-36   3.44519178e-36   2.62101

In [12]:
import _Sparse_Turbo_lib
%time _Sparse_Turbo_lib.FSM_update_sparse_test(in1_inds, in1_vals, c1_inds, c1_vals, S.F.S, S.F.Q, S.F.S1, S.F.Q1, S.F.Qm1,16 , 4096, 16)
#print in1_inds, in1_vals, c1_inds, c1_vals


CPU times: user 2.66 s, sys: 4 ms, total: 2.66 s
Wall time: 2.66 s


In [30]:
temp = np.vstack([VN2_update(out1_inds[i,:], b_inds[i,:], out1_vals[i,:], b_vals[i,:]) for i in range(k)])
in2_inds = temp[::2]
in2_vals = temp[1::2]

In [34]:
np.int64(in2_inds)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       [ 0,  1,  2,  5,  6,  9, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4,  5,  6,  7, 11, 14, 15, -1,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  9, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  9, 10, 13, 14, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  4, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [61]:
max_vals = 4
k = S.k
p = S.p
inter = S.inter
deinter = S.deinter
mbi = mu[:k,:]
mci = mu[k:,:]
map_inds_c1 = S.map_inds_c1
map_inds_c2 = S.map_inds_c2
mci1t = np.ones([k, Aq], float)/Aq;
mci1t[map_inds_c1, :] = mci[::2]
mci2t = np.ones([k, Aq], float)/Aq;
mci2t[map_inds_c2, :] = mci[1::2]


In [63]:
pc2 = mci2t.flatten()
pc1 = mci1t.flatten()
pd = mbi.flatten()

array([  1.95219570e-18,   5.11322739e-01,   1.47449293e-30,
         3.86201938e-13,   2.34503386e-14,   6.80569222e-29,
         9.74557338e-01,   2.82833327e-15])

In [1]:
run Advandeced_Turbo_lib.py

In [5]:
def QAM(M): # 2**M QAM constellation
    if M == 0:
        return np.array([0])
    odd = M % 2
    base = (M - odd) / 2
    min_Re = -2**(base+odd) + 1
    max_Re = 2**(base+odd) - 1
    min_Im = -2**base + 1
    max_Im = 2**base - 1
    const = np.asarray([i + 1j*j  for i in np.arange(min_Re, max_Re+1, 2)\
                                   for j in np.arange(min_Im, max_Im+1, 2)])
    scale = np.sqrt((np.abs(const)**2).sum()/float(2**M))
    return const / scale


Nb = 2
k = 5
const = QAM(Nb)
SNR = 12
Aq = 2**Nb


In [6]:
SNR = 5
S = Sparse_Turbo_Coder_MS(k, 5, 1./3, Aq, [], [])

In [7]:
#b = np.random.randint(Aq, size=k)
b = np.array([1,3,2,0,3],int)
c = S.encode(b)
print c
N = len(c)
print len(c)
s = const[c]
(x, sigma2w) = AWGN(s, SNR)

[1 3 2 0 3 1 3 0 3 1 2 3 0 3 3]
15


In [5]:
mu = np.zeros([N, Aq])
for i in range(Aq):
    mu[:, i] = np.exp(-np.abs(x - const[i])**2/sigma2w)

In [6]:
Nc = Aq
Nd = Aq
Ns = Aq**3
theta = 1e-8

In [7]:
max_vals = 4
k = S.k
p = S.p
inter = S.inter
deinter = S.deinter
mbi = mu[:k,:]
mci = mu[k:,:]
map_inds_c1 = S.map_inds_c1
map_inds_c2 = S.map_inds_c2
mci1t = -np.ones([k, Aq], float);
mci1t[map_inds_c1, :] = mci[::2]
mci2t = -np.ones([k, Aq], float);
mci2t[map_inds_c2, :] = mci[1::2]

pc1 = mci1t.flatten()
pc2 = mci2t.flatten()
pd = mbi.flatten()
res = np.zeros(k, int)
inter = S.inter
deinter = S.deinter

In [ ]:
_Sparse_Turbo_lib.turbo_decodeMS_func(pc1, pc2, pd, res, inter, deinter, S.K, S.k, Aq, Nc, Nd, Ns, theta)

In [1]:
run Advandeced_Turbo_lib.py

In [2]:
import _Turbo_lib
import _Sparse_Turbo_lib

In [19]:
N = 4
Aq = 2
d = np.random.randint(Aq,size=N)
c = np.zeros(N,int)
_Sparse_Turbo_lib.FSM_sparse_encode_MS(d, c, Aq)
SNR = 5


In [20]:
print c,d

[0 1 0 0] [0 1 1 1]


In [21]:
def ASK_MOD(Aq):
    Am = np.linspace(-1,1,Aq)
    return Am / np.sqrt(np.sum(Am**2)/Aq)

In [22]:
const = ASK_MOD(Aq)
s = np.hstack([const[d], const[c]])
(x, sigma2w) = AWGN(s, SNR)

In [23]:
mud = np.zeros([N, Aq], float)
muc = np.zeros([N, Aq], float)
pout = np.zeros([N, Aq], float)
Rxx = np.zeros([Aq,Aq])
for i in range(Aq):
    mud[:,i] = np.exp(-np.abs(x[:N] - const[i])**2/sigma2w)
    muc[:,i] = np.exp(-np.abs(x[N:] - const[i])**2/sigma2w)

In [18]:
_Sparse_Turbo_lib.FSM_update_eff(mud, muc, pout,1e-3)

0 0 0 0 0 0 (5, 8)
0 0 0 1 1 0 (5, 8)
lost track
0 0 0 0 0 0 (5, 8)
0 1 0 0 0 2 (5, 8)
1 0 1 0 0 5 (5, 8)
1 1 1 0 0 7 (5, 8)
[[ 1.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 1.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 1.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.125  0.125  0.125  0.125  0.125  0.125  0.125  0.125]
 [ 0.25   0.     0.25   0.     0.     0.25   0.     0.25 ]]


In [17]:
%time _Sparse_Turbo_lib.FSM_update_ref_simple_Rxx(mud, muc, pout, Rxx)

CPU times: user 35.3 s, sys: 129 ms, total: 35.4 s
Wall time: 35.4 s


In [18]:
np.sum(np.argmax(pout, axis=1) != d)

4127

In [19]:
v,w = np.linalg.eig(Rxx)

In [24]:
#plt.plot(np.sort(v),'r-')
plt.plot(w[np.argsort(v)[-1],:],'k-')
plt.show()

In [24]:
mudf = mud.flatten()
mucf = muc.flatten()
poutf = np.zeros(len(mudf))
S = np.zeros([Aq,Aq**3], int)
Q = np.zeros([Aq,Aq**3], int)
_Turbo_lib.create_matrix_SQ_MS_mod(S,Q)

In [25]:
_Turbo_lib.update_general_FSM_func(mucf, mudf, poutf, S, Q)

In [11]:
np.max(poutf.reshape(N,Aq)-pout)

0.00062305218272173946

In [32]:
Rxx = np.zeros([Aq,Aq])
for i in range(N):
    vec = np.asarray([mud[i,:]])
    Rxx += np.dot(vec.T,vec)/float(N)


In [33]:
v,w = np.linalg.eig(Rxx)

In [39]:
#plt.plot(np.sort(v),'r-')
plt.plot(w[:,np.argsort(v)[-4]],'k-')
plt.show()

In [ ]:
_Turbo_lib.update_FSM_func